In [17]:
import pandas as pd
import numpy as np

In [18]:
def list_of_chars(input:str)->list:
    '''Splits an AoC input into respective lines, and then splits 
    each line into a list of characters

    Args:
      input: str
        The AoC input as a string
    
    Returns:
      a list where each element is a line in the input
    '''
    input = input.splitlines()
    for idx, inp in enumerate(input):
        input[idx] = [char for char in inp]

    return input

In [19]:
# opening the file in read mode 
my_file = open("input.txt", "r") 
  
# reading the file 
data = my_file.read().split('\n')
data_list = [[char for char in row] for row in data]

In [21]:
test_str = """.|...\\....
|.-.\\.....
.....|-...
........|.
..........
.........\\
..../.\\\\..
.-.-/..|..
.|....-|.\\
..//.|...."""

In [22]:
test = list_of_chars(test_str)

In [23]:
test

[['.', '|', '.', '.', '.', '\\', '.', '.', '.', '.'],
 ['|', '.', '-', '.', '\\', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '|', '-', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '|', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '\\'],
 ['.', '.', '.', '.', '/', '.', '\\', '\\', '.', '.'],
 ['.', '-', '.', '-', '/', '.', '.', '|', '.', '.'],
 ['.', '|', '.', '.', '.', '.', '-', '|', '.', '\\'],
 ['.', '.', '/', '/', '.', '|', '.', '.', '.', '.']]

In [24]:
input_data = data_list
start_row = 0
start_col = 0
direction = "Right"
energised_dict = {}

In [42]:
def advance_beam(row, col, direction, input_data, energised_dict):
    """
    move the beam forward based on the current direction and the character faced

        :param int row: what is the current row position
        :param int col: what is the current col position
        :param str direction: what is the current direction
        :param list of lists input_data: dataset of chars split into list of lists
        :param dict energised_dict: dictionary of all coordinates and direction it was crossed in - utilised once for each start coord

        :returns dict energised_dict: dictionary of all coordinates and direction it was crossed in - utilised once for each start coord
    """

    if (row >= 0) & (row <= len(input_data)-1) & (col >= 0) & (col <= len(input_data[0])-1):
        char = input_data[row][col]
        #print(row, col, char)
        if (row, col) in energised_dict:
            if energised_dict[(row, col)] == direction:
                print(f"Route repeated at {row}, {col} with direction of {direction}")
                stop = True
            else:
                stop = False
        else:
            stop = False
        if stop == False:
            energised_dict[(row, col)] = direction
            if direction == "Right":
                if char == "\\":
                    advance_beam(row+1, col, "Down", input_data, energised_dict)
                if (char == "-") | (char == "."):
                    advance_beam(row, col+1, "Right", input_data, energised_dict)
                if char == "/":
                    advance_beam(row-1, col, "Up", input_data, energised_dict)
                if char == "|":
                    advance_beam(row-1, col, "Up", input_data, energised_dict)
                    advance_beam(row+1, col, "Down", input_data, energised_dict)
            elif direction == "Left":
                if char == "\\":
                    advance_beam(row-1, col, "Up", input_data, energised_dict)
                if (char == "-") | (char == "."):
                    advance_beam(row, col-1, "Left", input_data, energised_dict)
                if char == "/":
                    advance_beam(row+1, col, "Down", input_data, energised_dict)
                if char == "|":
                    advance_beam(row-1, col, "Up", input_data, energised_dict)
                    advance_beam(row+1, col, "Down", input_data, energised_dict)
            elif direction == "Up":
                if char == "\\":
                    advance_beam(row, col-1, "Left", input_data, energised_dict)
                if char == "-":
                    advance_beam(row, col-1, "Left", input_data, energised_dict)
                    advance_beam(row, col+1, "Right", input_data, energised_dict)
                if char == "/":
                    advance_beam(row, col+1, "Right", input_data, energised_dict)
                if (char == "|") | (char == "."):
                    advance_beam(row-1, col, "Up", input_data, energised_dict)
            elif direction == "Down":
                if char == "\\":
                    advance_beam(row, col+1, "Right", input_data, energised_dict)
                if char == "-":
                    advance_beam(row, col-1, "Left", input_data, energised_dict)
                    advance_beam(row, col+1, "Right", input_data, energised_dict)
                if char == "/":
                    advance_beam(row, col-1, "Left", input_data, energised_dict)
                if (char == "|") | (char == "."):
                    advance_beam(row+1, col, "Down", input_data, energised_dict)
    else:
        print(f"Beam left grid at {row}, {col}")
    
    return energised_dict

In [50]:
def get_num_energised(row, col, direction):
    """
    input a start point and direction and count the num energised tiles for that config

        :param int row: what is the current row position
        :param int col: what is the current col position
        :param str direction: what is the current direction

        :return str config: Concatenated string of row start,col start and direction start
        :return int num_energised: Sum of energised nodes for this config
    """
    print(f"Running for {row},{col} start in {direction} direction")
    energised_dict = {}
    #print(energised_dict)
    temp_data = data_list.copy()
    energised_dict = advance_beam(row, col, direction, temp_data, energised_dict)
    #print(energised_dict)
    num_energised = len(set(energised_dict.keys()))
    config = f"{row}_{col}_{direction}"

    return config, num_energised   

In [51]:
# Set config min and max values
min_row = 0
min_col = 0
max_row = len(data_list)
max_col = len(data_list[0])

In [52]:
"""loop over all cols and check energised list for min and max row,
then loop over all rows and check energised list for min and max cols.
Append all results and the config to a summary dict"""
summary_dict = {}
for i in range(min_row, max_row):
    config, num_energised = get_num_energised(i, min_col, "Right")
    summary_dict[config] = num_energised
    config, num_energised = get_num_energised(i, max_col, "Left")
    summary_dict[config] = num_energised
for i in range(min_col, max_col):
    config, num_energised = get_num_energised(min_row, i, "Down")
    summary_dict[config] = num_energised
    config, num_energised = get_num_energised(max_row, i, "Up")
    summary_dict[config] = num_energised

Running for 0,0 start in Right direction
Beam left grid at -1, 13
Beam left grid at 20, -1
Beam left grid at -1, 4
Beam left grid at 103, -1
Beam left grid at 110, 2
Beam left grid at 26, -1
Beam left grid at -1, 72
Beam left grid at -1, 21
Beam left grid at -1, 64
Beam left grid at -1, 87
Beam left grid at -1, 66
Beam left grid at 64, 110
Route repeated at 67, 72 with direction of Up
Route repeated at 77, 79 with direction of Up
Beam left grid at 97, 110
Route repeated at 79, 82 with direction of Right
Route repeated at 48, 71 with direction of Left
Route repeated at 20, 64 with direction of Up
Route repeated at 49, 106 with direction of Left
Beam left grid at 49, 110
Beam left grid at 110, 73
Route repeated at 10, 73 with direction of Up
Beam left grid at -1, 45
Route repeated at 9, 43 with direction of Up
Route repeated at 7, 21 with direction of Up
Route repeated at 21, 63 with direction of Right
Beam left grid at 44, -1
Beam left grid at -1, 63
Route repeated at 62, 38 with direct

In [54]:
# Find the max value from the summary list
max(summary_dict.values())

8089

In [55]:
# Find the config based on the max value in the summary list
print(list(summary_dict.keys())[list(summary_dict.values()).index(max(summary_dict.values()))])

0_83_Down
